In [1]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession



In [2]:
spark = SparkSession.builder.appName("IP logging").getOrCreate()

In [3]:
web_log_df = spark.read.csv('web_server_logs.csv', header=True, inferSchema=True)

In [4]:
web_log_df.show(5)

+--------------+-------------------+------+--------------------+-------------+-------------+
|            ip|          timestamp|method|                 url|response_code|response_size|
+--------------+-------------------+------+--------------------+-------------+-------------+
|21.104.147.251|2025-02-02 17:13:19|   PUT|     search/category|          404|         4336|
|   79.55.85.39|2025-02-12 11:40:16|  POST|      blog/list/blog|          200|         6093|
|146.23.151.150|2025-02-24 18:54:21|DELETE|           main/list|          200|         2327|
| 214.156.65.56|2025-02-10 09:33:29|   PUT|category/app/explore|          404|          966|
|  161.65.229.8|2025-02-16 01:26:03|   GET|explore/blog/wp-c...|          200|         3286|
+--------------+-------------------+------+--------------------+-------------+-------------+
only showing top 5 rows



1. Сгруппируйте данные по IP и посчитайте количество запросов для каждого IP, выводим 10 самых активных IP. Формат вывода, как на скрине ниже.

In [5]:
print('Top 10 IP addreses:')
web_log_df.groupBy('ip').agg(F.count('url').alias('request_count')).sort(F.col('request_count').desc()).limit(10).show()

Top 10 IP addreses:
+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
|  96.248.127.24|            1|
|  63.88.199.227|            1|
|   89.55.52.192|            1|
| 170.227.53.254|            1|
|  122.41.128.64|            1|
| 121.201.93.250|            1|
| 158.195.171.53|            1|
|189.127.112.164|            1|
|   24.25.233.55|            1|
|  103.47.113.83|            1|
+---------------+-------------+



2. Сгруппируйте данные по HTTP-методу и посчитайте количество запросов для каждого метода

In [6]:
print('Request count by HTTP method:')
web_log_df.groupBy('method').agg(F.count('url').alias('method_count')).sort(F.col('method_count').desc()).limit(10).show()

Request count by HTTP method:
+------+------------+
|method|method_count|
+------+------------+
|   GET|       25153|
|  POST|       25042|
|DELETE|       24928|
|   PUT|       24877|
+------+------------+



3. Профильтруйте и посчитайте количество запросов с кодом ответа 404.

In [7]:
response_404 = web_log_df.filter(F.col('response_code') == 404).count()

In [8]:
print(f'Number of 404 responsecodes: {response_404}')

Number of 404 responsecodes: 25014


4. Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате.

In [9]:
web_log_df = web_log_df.withColumn("date", F.to_date(F.col("timestamp"), "yyyy-MM-dd"))

In [10]:
web_log_df.show(5)

+--------------+-------------------+------+--------------------+-------------+-------------+----------+
|            ip|          timestamp|method|                 url|response_code|response_size|      date|
+--------------+-------------------+------+--------------------+-------------+-------------+----------+
|21.104.147.251|2025-02-02 17:13:19|   PUT|     search/category|          404|         4336|2025-02-02|
|   79.55.85.39|2025-02-12 11:40:16|  POST|      blog/list/blog|          200|         6093|2025-02-12|
|146.23.151.150|2025-02-24 18:54:21|DELETE|           main/list|          200|         2327|2025-02-24|
| 214.156.65.56|2025-02-10 09:33:29|   PUT|category/app/explore|          404|          966|2025-02-10|
|  161.65.229.8|2025-02-16 01:26:03|   GET|explore/blog/wp-c...|          200|         3286|2025-02-16|
+--------------+-------------------+------+--------------------+-------------+-------------+----------+
only showing top 5 rows



In [11]:
print('Total response size by day:')
web_log_df.groupBy('date').agg(F.sum('response_size').alias('total_response_size')).sort('date').show()

Total response size by day:
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2025-01-01|            7615124|
|2025-01-02|            7582730|
|2025-01-03|            7566740|
|2025-01-04|            7388726|
|2025-01-05|            7383785|
|2025-01-06|            7112499|
|2025-01-07|            7448674|
|2025-01-08|            7216101|
|2025-01-09|            7118227|
|2025-01-10|            7405951|
|2025-01-11|            7186691|
|2025-01-12|            7465617|
|2025-01-13|            7732821|
|2025-01-14|            7323256|
|2025-01-15|            7708653|
|2025-01-16|            7174167|
|2025-01-17|            7331699|
|2025-01-18|            7440824|
|2025-01-19|            7732956|
|2025-01-20|            7384204|
+----------+-------------------+
only showing top 20 rows

